<div class="info">
I'm working my way through the points in this <a href="https://medium.com/@steCoersionphenthecurt/33-fundamentals-every-javascript-developer-should-know-13dd720a90d1" rel="external" target="_blank">list of 33 fundamentals every JavaScript developer should know</a> in order to make sure I understand the language thoroughly. 
    
This notebook is for working through code relating to point 6.
</div>

# Scope and Context

- When we talk about **scope** we are talking about the variables a piece of code has access to at runtime.
- When we talk about **context** we are talking about the value of `this` and all of it's properties.

# `var` is bananas

Variables defined in the default or root scope are accessible globally, and variables defined within functions are only accessible within that function.

In [1]:
var a = 'apples'

function fruitVendor() {
    var b = 'bananas'
    console.log('We are inside the scope of the fruitVendor function:')
    console.log('> a is for ' + a)
    console.log('> b is for ' + b)
    console.log('\n')
}

fruitVendor()

console.log('We are at the default or root scope level:')
console.log('> a is for ' + a)
console.log('> b is for ' + b)

We are inside the scope of the fruitVendor function:
> a is for apples
> b is for bananas


We are at the default or root scope level:
> a is for apples


ReferenceError: b is not defined

We get **scope conflict** when there is a variable of the same name in the parent and child scope. 

In [2]:
var a = 'apples'

function fruitVendor() {
    var a = 'avocados'
    console.log('We are inside the scope of the fruitVendor function:')
    console.log('> a is for ' + a)
    console.log('\n')
}

fruitVendor()

console.log('We are at the default or root scope level:')
console.log('> a is for ' + a)

We are inside the scope of the fruitVendor function:
> a is for avocados


We are at the default or root scope level:
> a is for apples


Although the code runs, the problem is that we no longer have access to the parent variable of the same name due to naming conflicts.

**This is not really new information. But we are going somewhere with this which is JS specific so bear with me.**

Above, we looked at function scope, but what about block scope? Is there much difference here when we try to define variables inside `if`, `for`, or `while` blocks of code?

In [3]:
if (true) {
    var c = 'clementine'
}

console.log('> c is for ' + c)

> c is for clementine


`var` values can be accessed outside of the scope of the block in which they were defined. This is considered by some to be inconsistent and confusing, particularly because you might feel that scope could be imagined as being encapsulated in curly braces. Lack of block scope isn't something you'd be used to if coming from many other programming languages. So in ES6 a new pair of definitions were introduced....

# `let` and `const`

In [4]:
let d = 'durians'

if (true) {
    let e = 'elderberries'
    console.log('We are inside the scope of the if block:')
    console.log('> d is for ' + d)
    console.log('> e is for ' + e)
    console.log('\n')
}

console.log('We are at the default or root scope level:')
console.log('> d is for ' + d)
console.log('> e is for ' + e)

We are inside the scope of the if block:
> d is for durians
> e is for elderberries


We are at the default or root scope level:
> d is for durians


ReferenceError: e is not defined

Now block scope is operating the same as function scope and we can all sleep soundly at night. Hooray.

# Getting Loopy

So we know that `let` and `var` operate differently in terms of scope. What happens if we use `var` when adding lazy loaded functions to an array, executing them at some later time?

In [5]:
var toBeExecuted = []

for (var i=0; i<3; i++) {  
  toBeExecuted.push(() => console.log(i))
}

toBeExecuted.forEach(lazyFn => lazyFn())

3
3
3


What happened? Instead of creating a local variable `i` for each increment in the loop, it ended up printing the final value for that variable for all function calls.

It feels unexpected and although there are work-arounds for still using a `var` they can be a little verbose.

Enter the `let` keyword:

In [6]:
let letItBeExecuted = []

for (let i=0; i<3; i++) {  
  letItBeExecuted.push(() => console.log(i))
}

letItBeExecuted.forEach(lazyFn => lazyFn())

0
1
2


Magic ✨

# A Warning 

It's important to note that JS still compiles when you create a new variable, even if you don't specify `var`, `const`, or `let`. When you do so, it first searches the current scope for a variable of that name, then trickles up through parents layers. 

⚠️ If it doesn't find it all the way up to the root, it will create the variable there in the root layer, also known as a global variable. This is known as **"polluting the global scope"**.

To avoid this place the string `"use strict"` at the top of the entry point file, causing a `ReferenceError` message if this is attempted. [This trick solves a whole host of other problems as well.](https://www.w3schools.com/js/js_strict.asp)

In [7]:
"use strict"
function badFunction() {
    w = 'watermelon'
}

console.log(w)

ReferenceError: w is not defined

# Digging Deeper

JavaScript allows for nesting functions, and for passing functions aroung like variables. So what happens when nested functions take advantage of their parent scope variables before being passed into and executed by the root layer (or any layer outside of that scope) where these variables should not be accessible?

In [11]:
function veggieVendor() {
    var a = 'aparagus'
    
    function innerFunction() { 
        console.log('a is for ' + a)
    }
    
    return innerFunction
}

var printVeggie = veggieVendor()
printVeggie()

a is for aparagus


This example is taking advantage of the **lexical scope** model used by JavaScript. You can think of lexical scope as being "when identifiers are scoped to functions".

To try to understand what this is or how this happens we need to dig a little deeper into the mechanics of JavaScript. As, although it is referred to as an *interpreted* language, JavaScript does compile code immediately before executing it, nad for this reason it can also be called a *just-in-time compiled* language. 

The **lexing phase** of compilation finds where and how all identifiers are declared, and how they will be looked up during execution. So the JavaScript engine is aware of these before execution.

For example the statement: `var a = "apple"` is split into two separate steps at lexing time:

   - `var a`       This declares the variable in the scope, before code execution.
   - `a = "apple"` This assigns the string value to the variable a, if it is found within the available scope.
